#Introduce

This notebook is created for the case using to load the Dataset (DIV2k) with difference key, and preprocessing image:


1.   Load the dataset with key.
2.   Preprocessing image.
3.   Seperate the dataset.



In [ ]:
print('import libary...')
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
  

import libary...


In [ ]:
def load_dataset(key='div2k',include_validate=True):
  print('using tensorflow_datasets to load the dataset','\nloading dataset...')
  if include_validate:
    (train_ds, val_ds), metadata = tfds.load(
    key,
    split=['train','validation'],
    with_info=True,
    as_supervised=True,
   )
  else:
    (train_ds, val_ds), metadata = tfds.load(
    key,
    split=['train'],
    with_info=True,
    as_supervised=True,
    )
  print(metadata)
  return (train_ds,val_ds),metadata

In [ ]:
def normalize_img(image):
  print('Normalizes images: `uint8` -> `float32`')
  return tf.cast(image, tf.float32) /255

In [ ]:
def configure_performan(ds,BUFFER_SIZE=1000,BATCH_SIZE=4):
  ds=ds.cache()
  ds=ds.shuffle(buffer_size=BUFFER_SIZE)
  ds=ds.batch(BATCH_SIZE)
  ds=ds.prefetch(buffer_size=tf.data.AUTOTUNE)
  return ds

In [ ]:
(rain_ds,val_ds),infor=load_dataset('div2k/bicubic_x4')

In [ ]:

len(rain_ds)

800

In [ ]:
sub_data=rain_ds.take(40)

In [ ]:
for i in sub_data.take(1):
  print(i[0])
  # print(i[1])

In [ ]:

def show(image, label):
  plt.figure(figsize=(len(image.numpy())/80,len(image.numpy()[0])/80))
  plt.imshow(image)
  plt.title(np.array(label))
  plt.axis('off')

In [ ]:
for i in sub_data.take(10):
    # print(len(i[0].numpy()))
  show(i[0],'Low resolution')
  show(i[1],'High resolution')

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
for i in sub_data.take(1):
  print(i[1].numpy())  

In [ ]:
sub_data=sub_data.map(lambda lr,hr: (normalize_img(lr) , normalize_img(hr)),num_parallel_calls=tf.data.AUTOTUNE)

Normalizes images: `uint8` -> `float32`
Normalizes images: `uint8` -> `float32`


In [ ]:
len(sub_data)

40

In [ ]:
def visualize_augmented(original, augmented,type='lr'):
  # plt.figure(figsize=(len(original.numpy())/80,len(original.numpy()[0])/80))
  fig = plt.figure()
  plt.subplot(1,2,1)
  plt.title('Original image'+type)
  plt.imshow(original)
  plt.subplot(1,2,2)
  plt.title('Augmented image'+type)
  plt.imshow(augmented)

In [ ]:
g=tf.random.get_global_generator()

In [ ]:
def augment_sr(lr,hr):
  #get the same seed to get the same operator to both lr and hr for image when we use randomly
  seed= tf.random.get_global_generator().make_seeds(2)[0]
  #low resolution augmentation
  lr=tf.image.stateless_random_contrast(lr,0.8,1.2,seed)
  lr=tf.image.stateless_random_flip_left_right(lr,seed)
  lr=tf.image.stateless_random_brightness(lr,0.15,seed)
  lr=tf.image.stateless_random_saturation(lr,0.8,1.3,seed)
  #high resolution augmentation
  hr=tf.image.stateless_random_contrast(hr,0.8,1.2,seed)
  hr=tf.image.stateless_random_flip_left_right(hr,seed)
  hr=tf.image.stateless_random_brightness(hr,0.15,seed)
  hr=tf.image.stateless_random_saturation(hr,0.8,1.3,seed)
  return lr,hr

In [ ]:

for i in sub_data.take(6):
  au1,au2=augment(i[0],i[1])
  visualize(i[0],au1,'lr')
  visualize(i[1],au2,type='hr')
  # show(i[1],'High resolution')


In [ ]:
sub_data=sub_data.map(augment,num_parallel_calls=tf.data.AUTOTUNE)


In [ ]:
sub_data=sub_data.shuffle(buffer_size=40)